<a href="https://colab.research.google.com/github/abdullah-saridag/Machine-Learning-Bootcamp/blob/main/MLBootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regresyon projemiz için ihtiyaç duyduğum kütüphaneleri ekliyorum

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sn
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV

Kaggle dan aldığım veri setini drive dan colaba yükleyip zip dosyasını açıp daha sonra siliyorum

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle Veri Seti for Colab"
%cd "/content/gdrive/MyDrive/Kaggle Veri Seti for Colab"
!kaggle datasets download -d budincsevity/szeged-weather
!unzip \*.zip && rm*.zip

Veri setimi dt değişkenine atadım ve veri setimi okuyorum 96453 rows × 12 columns dan oluşuyor

In [ ]:
dt = pd.read_csv('weatherHistory.csv')
dt

Veriyi nümerik olarak özetliyorum

In [ ]:
dt.info()

İnfo ile aldığımız genel bilgiye ait başka bir özet

In [ ]:
dt.describe()

Sıcaklık dağılımını gösteren histogram çizdirdim

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(dt ['Temperature (C)'], bins=30, kde=True, color='blue')
plt.xlabel('Sıcaklık (C)')
plt.ylabel('Frekans')
plt.show()

Nem ve rüzgar hızı arasındaki ilişkiyi veren histogram

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Humidity', y='Wind Speed (km/h)', data=dt, alpha=0.5)
plt.xlabel('Nem')
plt.ylabel('Rüzgar Hızı (km/h)')
plt.show()

Nemin KDE grafiği

In [ ]:
sns.kdeplot(dt['Humidity'], fill=True, color='blue')
plt.xlabel('Humidity')
plt.ylabel('Density')
plt.title('Humidity Distribution (KDE)')
plt.show()

Veri seti temizliği (eksik değerler içeren satırları silme, gereksiz sütünları kaldırma) ve normalleştirme yaptım.

In [7]:
dt.dropna(inplace=True)
dt.drop(['Daily Summary', 'Loud Cover'], axis=1, inplace=True)
scaler = StandardScaler()
dt[['Temperature (C)', 'Humidity', 'Wind Speed (km/h)']] = scaler.fit_transform(dt[['Temperature (C)', 'Humidity', 'Wind Speed (km/h)']])

Label encoding ve one hot encoding yaptım

In [8]:
label_encoder = LabelEncoder()
dt['Precip Type'] = label_encoder.fit_transform(dt['Precip Type'])
dt = pd.get_dummies(dt, columns=['Summary'])


Veri setini eğitim ve test kümelerine ayırdım %80 eğitim %20 test

In [ ]:
X = dt.drop(['Temperature (C)'], axis=1)
y = dt['Temperature (C)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim kümesinin boyutu:", X_train.shape)
print("Test kümesinin boyutu:", X_test.shape)

Bağımlı ve bağımsız değişkenleri ayırdım bağımsız değişken nem bağımlı sıcaklık görünen sıcaklık.
modelleri oluşturdum
modelleri eğittim ve test ettim
Modellerin K-fold çapraz doğrulama ile R2 değerini Cross-Validation Scoru ve MSE sini hesapladım


In [ ]:
X = dt[['Humidity']]
y = dt['Apparent Temperature (C)']

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'XGBoost Regression': XGBRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'K-Nearest Neighbors Regression': KNeighborsRegressor(n_neighbors=311),
}

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    print(f"{name} R² Scores:", scores)
    print(f"{name} Mean R² Score:", scores.mean())

    print(f"{name} Cross-Validation Scores:", scores)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} Mean Squared Error:", mse)

Çıkan sonuçlar neticesinde Random Forest Regression modelini daha başarılı gördüm ve Randomized Search yöntemini kullanacağım
Parametreleri tanımladım modeli oluşturdum en iyi Hiperparametreleri buldum ve yazdırdım.

In [ ]:
param_dist = {
    'n_estimators': randint(10, 100),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

rf = RandomForestRegressor()

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

print("En iyi hiperparametreler:", random_search.best_params_)

best_model = random_search.best_estimator_

Bulduğum en iyi hiperparametreleri aldım en iyi modeli oluşturdum modeli eğittim Tahminleri yaptırdım ve performansını değerlerdirdim

In [13]:
# En iyi hiperparametreler
best_params = {'max_depth': 12, 'min_samples_leaf': 4, 'min_samples_split': 7, 'n_estimators': 71}

# En iyi modelin oluşturulması
best_model = RandomForestRegressor(**best_params)

# Modelin eğitimi
best_model.fit(X_train, y_train)

# Tahminlerin yapılması
y_pred = best_model.predict(X_test)


# Performansın değerlendirilmesi
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

Mean Squared Error: 68.17023447975917
Mean Absolute Error: 6.63758059219255
